# Goal : To predict Diabetes using only Apple watch data



In [2]:
import pandas as pd
df = pd.read_xml("/Users/nilvaghela/Desktop/Research/Diabetes/Data/export.xml")

In [5]:
df.head(-1)

,value,HKCharacteristicTypeIdentifierDateOfBirth,HKCharacteristicTypeIdentifierBiologicalSex,HKCharacteristicTypeIdentifierBloodType,HKCharacteristicTypeIdentifierFitzpatrickSkinType,HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse,type,sourceName,sourceVersion,unit,...,activeEnergyBurned,activeEnergyBurnedGoal,activeEnergyBurnedUnit,appleMoveTime,appleMoveTimeGoal,appleExerciseTime,appleExerciseTimeGoal,appleStandHours,appleStandHoursGoal,HeartRateVariabilityMetadataList
0,2025-11-28 22:21:13 -0500,None,None,None,None,NaN,None,None,None,None,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
1,None,1999-04-06,HKBiologicalSexMale,HKBloodTypeNotSet,HKFitzpatrickSkinTypeNotSet,NaN,None,None,None,None,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
2,250,None,None,None,None,NaN,HKQuantityTypeIdentifierDietaryWater,Diet,530,mL,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
3,5.51181,None,None,None,None,NaN,HKQuantityTypeIdentifierHeight,iPhone,12.3.1,ft,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
4,5.18373,None,None,None,None,NaN,HKQuantityTypeIdentifierHeight,Nilrajsinh vaghela’s iPhone,12.3,ft,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2617045,44.7817,None,None,None,None,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Nilrajsinh’s Apple Watch,26.1,ms,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,\n
2617046,26.0156,None,None,None,None,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Nilrajsinh’s Apple Watch,26.1,ms,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,\n
2617047,18.7718,None,None,None,None,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Nilrajsinh’s Apple Watch,26.1,ms,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,\n
2617048,26.0455,None,None,None,None,NaN,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Nilrajsinh’s Apple Watch,26.1,ms,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,\n


In [16]:
from pathlib import Path

test_path = Path("/Users/nilvaghela/Desktop/Research/Diabetes/Data/export.xml")
print("XML exists?:", test_path.exists())

folder_path = Path("/Users/nilvaghela/Desktop/Research/Diabetes/Data/")
print("Folder exists?:", folder_path.exists())


XML exists?: True
Folder exists?: True


In [18]:
import xml.etree.ElementTree as ET
import csv
from pathlib import Path

xml_path = Path("/Users/nilvaghela/Desktop/Research/Diabetes/Data/export.xml")
csv_path = xml_path.with_name("health_records.csv")

print("Reading:", xml_path)
print("Writing:", csv_path)

# Start with empty header
fieldnames = set()

rows_buffer = []

# First pass: collect ALL fieldnames
for event, elem in ET.iterparse(xml_path):
    if elem.tag == "Record":
        for key in elem.attrib.keys():
            fieldnames.add(key)
        rows_buffer.append(elem.attrib.copy())  # store row
        elem.clear()

fieldnames = sorted(list(fieldnames))
print("Columns found:", fieldnames)

# Second pass: write CSV with complete header
with open(csv_path, "w", newline="") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for row in rows_buffer:
        writer.writerow(row)

print("DONE — health_records.csv created!")
print("Path:", csv_path)


Reading: /Users/nilvaghela/Desktop/Research/Diabetes/Data/export.xml
Writing: /Users/nilvaghela/Desktop/Research/Diabetes/Data/health_records.csv
Columns found: ['creationDate', 'device', 'endDate', 'sourceName', 'sourceVersion', 'startDate', 'type', 'unit', 'value']
DONE — health_records.csv created!
Path: /Users/nilvaghela/Desktop/Research/Diabetes/Data/health_records.csv


In [19]:
import pandas as pd

df = pd.read_csv("/Users/nilvaghela/Desktop/Research/Diabetes/Data/health_records.csv")
df.head()
df.columns


/var/folders/ph/wydd_qxn59v1_1ynz85860l00000gn/T/ipykernel_26906/3714213657.py:3: DtypeWarning: Columns (1,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/nilvaghela/Desktop/Research/Diabetes/Data/health_records.csv")


Index(['creationDate', 'device', 'endDate', 'sourceName', 'sourceVersion',
       'startDate', 'type', 'unit', 'value'],
      dtype='object')

In [21]:
import pandas as pd
import numpy as np
from pathlib import Path

# ---------- Paths ----------
base_dir = Path("/Users/nilvaghela/Desktop/Research/Diabetes/Data")
records_path = base_dir / "health_records.csv"

print("Loading:", records_path)
df = pd.read_csv(records_path)

# Ensure datetime
df["startDate"] = pd.to_datetime(df["startDate"])
df["endDate"]   = pd.to_datetime(df["endDate"], errors="ignore")

# ---------- Helper to extract a single metric ----------
def extract(type_id, col_name, unit_filter=None):
    """
    Extract one metric from health_records by HealthKit 'type' (and optional 'unit'),
    returning a DataFrame with columns: ['startDate', col_name].
    """
    temp = df[df["type"] == type_id].copy()
    if temp.empty:
        print(f"[WARN] No rows for type: {type_id}")
        return pd.DataFrame(columns=["startDate", col_name])

    if unit_filter is not None:
        temp = temp[temp["unit"] == unit_filter].copy()
        if temp.empty:
            print(f"[WARN] No rows for type: {type_id} with unit={unit_filter}")
            return pd.DataFrame(columns=["startDate", col_name])

    temp = temp[["startDate", "value"]].copy()
    temp.rename(columns={"value": col_name}, inplace=True)
    temp[col_name] = pd.to_numeric(temp[col_name], errors="coerce")
    return temp


# ---------- Core metrics you already had ----------
heart_rate = extract("HKQuantityTypeIdentifierHeartRate", "heart_rate", "count/min")
hrv        = extract("HKQuantityTypeIdentifierHeartRateVariabilitySDNN", "hrv", "ms")
steps      = extract("HKQuantityTypeIdentifierStepCount", "steps", "count")
resting_hr = extract("HKQuantityTypeIdentifierRestingHeartRate", "resting_hr", "count/min")
vo2        = extract("HKQuantityTypeIdentifierVO2Max", "vo2max", "mL/min·kg")

# ---------- Additional high-value metrics ----------
active_energy = extract("HKQuantityTypeIdentifierActiveEnergyBurned", "active_energy_kcal", "kcal")
basal_energy  = extract("HKQuantityTypeIdentifierBasalEnergyBurned", "basal_energy_kcal", "kcal")

distance_walk_run = extract("HKQuantityTypeIdentifierDistanceWalkingRunning",
                            "distance_walk_run_m", None)  # unit often "m"

flights = extract("HKQuantityTypeIdentifierFlightsClimbed", "flights", "count")

exercise_time = extract("HKQuantityTypeIdentifierAppleExerciseTime",
                        "exercise_minutes", "min")

stand_time = extract("HKQuantityTypeIdentifierAppleStandTime",
                     "stand_minutes", "min")

resp_rate = extract("HKQuantityTypeIdentifierRespiratoryRate",
                    "respiratory_rate", "count/min")

spo2 = extract("HKQuantityTypeIdentifierOxygenSaturation",
               "spo2_percent", "%")

wrist_temp = extract("HKQuantityTypeIdentifierBodyTemperature",
                     "wrist_temp_degC", "degC")


# ---------- Sleep (special handling) ----------
sleep = df[df["type"] == "HKCategoryTypeIdentifierSleepAnalysis"].copy()
if not sleep.empty:
    sleep["sleep_minutes"] = (
        pd.to_datetime(sleep["endDate"]) - pd.to_datetime(sleep["startDate"])
    ).dt.total_seconds() / 60
    sleep = sleep[["startDate", "sleep_minutes"]].rename(columns={"startDate": "timestamp"})
else:
    sleep = pd.DataFrame(columns=["timestamp", "sleep_minutes"])


# ---------- Merge everything into ONE DataFrame ----------
# Start with heart_rate as base (if empty, we fallback later)
metric_dfs = [
    heart_rate, hrv, steps, resting_hr, vo2,
    active_energy, basal_energy,
    distance_walk_run, flights,
    exercise_time, stand_time,
    resp_rate, spo2, wrist_temp
]

# Filter out completely empty ones
metric_dfs = [m for m in metric_dfs if not m.empty]

if not metric_dfs:
    raise ValueError("No metric dataframes found; check your health_records types.")

merged = metric_dfs[0]
for m in metric_dfs[1:]:
    merged = merged.merge(m, on="startDate", how="outer")

# Add sleep by merging on timestamp (same as startDate)
merged = merged.sort_values("startDate")
merged = merged.rename(columns={"startDate": "timestamp"})

# Merge sleep (outer join on timestamp)
merged = merged.merge(sleep, on="timestamp", how="left")

# ---------- Clean and save ----------
# Optional: forward/back fill to avoid big NaN gaps within sessions
merged = merged.sort_values("timestamp")
merged = merged.ffill().bfill()

out_path = base_dir / "apple_watch_combined_full.csv"
merged.to_csv(out_path, index=False)

print("\nCombined full CSV saved at:")
print(out_path)
print("\nColumns in final dataset:")
print(merged.columns.tolist())
print("\nPreview:")
print(merged.head())


Loading: /Users/nilvaghela/Desktop/Research/Diabetes/Data/health_records.csv


/var/folders/ph/wydd_qxn59v1_1ynz85860l00000gn/T/ipykernel_26906/2326300659.py:10: DtypeWarning: Columns (1,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(records_path)
/var/folders/ph/wydd_qxn59v1_1ynz85860l00000gn/T/ipykernel_26906/2326300659.py:14: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df["endDate"]   = pd.to_datetime(df["endDate"], errors="ignore")


[WARN] No rows for type: HKQuantityTypeIdentifierActiveEnergyBurned with unit=kcal
[WARN] No rows for type: HKQuantityTypeIdentifierBasalEnergyBurned with unit=kcal
[WARN] No rows for type: HKQuantityTypeIdentifierBodyTemperature

Combined full CSV saved at:
/Users/nilvaghela/Desktop/Research/Diabetes/Data/apple_watch_combined_full.csv

Columns in final dataset:
['timestamp', 'heart_rate', 'hrv', 'steps', 'resting_hr', 'vo2max', 'distance_walk_run_m', 'flights', 'exercise_minutes', 'stand_minutes', 'respiratory_rate', 'spo2_percent', 'sleep_minutes']

Preview:
                  timestamp  heart_rate      hrv  steps  resting_hr  vo2max  \
0 2018-02-10 05:37:30-05:00        79.0  132.015   13.0        70.0   34.25   
1 2018-02-10 06:34:33-05:00        79.0  132.015  355.0        70.0   34.25   
2 2018-02-10 06:44:32-05:00        79.0  132.015  157.0        70.0   34.25   
3 2018-02-10 06:54:19-05:00        79.0  132.015   24.0        70.0   34.25   
4 2018-02-10 07:07:11-05:00        79.

In [22]:
import pandas as pd
from pathlib import Path

base_dir = Path("/Users/nilvaghela/Desktop/Research/Diabetes/Data")
records_path = base_dir / "health_records.csv"
df = pd.read_csv(records_path)

# Look at all distinct types
print("Unique types containing 'Energy':")
print(df[df["type"].str.contains("Energy", na=False)][["type", "unit"]].drop_duplicates())

print("\nUnique types containing 'Temperature':")
print(df[df["type"].str.contains("Temperature", na=False)][["type", "unit"]].drop_duplicates())


/var/folders/ph/wydd_qxn59v1_1ynz85860l00000gn/T/ipykernel_26906/17843379.py:6: DtypeWarning: Columns (1,4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(records_path)


Unique types containing 'Energy':
                                               type unit
835842    HKQuantityTypeIdentifierBasalEnergyBurned  Cal
1166078  HKQuantityTypeIdentifierActiveEnergyBurned  Cal

Unique types containing 'Temperature':
                                                      type  unit
2374199  HKQuantityTypeIdentifierAppleSleepingWristTemp...  degF


In [23]:
import pandas as pd
from pathlib import Path

base_dir = Path("/Users/nilvaghela/Desktop/Research/Diabetes/Data")
records_path = base_dir / "health_records.csv"

df = pd.read_csv(records_path, low_memory=False)


In [24]:
import pandas as pd
import numpy as np
from pathlib import Path

# ---------- Paths ----------
base_dir = Path("/Users/nilvaghela/Desktop/Research/Diabetes/Data")
records_path = base_dir / "health_records.csv"

print("Loading:", records_path)
df = pd.read_csv(records_path, low_memory=False)

# Ensure datetime
df["startDate"] = pd.to_datetime(df["startDate"])
df["endDate"]   = pd.to_datetime(df["endDate"], errors="ignore")

# ---------- Helper to extract a single metric ----------
def extract(type_id, col_name, unit_filter=None):
    """
    Extract one metric from health_records by HealthKit 'type' (and optional 'unit'),
    returning a DataFrame with columns: ['startDate', col_name].
    """
    temp = df[df["type"] == type_id].copy()
    if temp.empty:
        print(f"[WARN] No rows for type: {type_id}")
        return pd.DataFrame(columns=["startDate", col_name])

    if unit_filter is not None:
        temp = temp[temp["unit"] == unit_filter].copy()
        if temp.empty:
            print(f"[WARN] No rows for type: {type_id} with unit={unit_filter}")
            return pd.DataFrame(columns=["startDate", col_name])

    temp = temp[["startDate", "value"]].copy()
    temp.rename(columns={"value": col_name}, inplace=True)
    temp[col_name] = pd.to_numeric(temp[col_name], errors="coerce")
    return temp


# ---------- Core metrics ----------
heart_rate = extract("HKQuantityTypeIdentifierHeartRate", "heart_rate", "count/min")
hrv        = extract("HKQuantityTypeIdentifierHeartRateVariabilitySDNN", "hrv", "ms")
steps      = extract("HKQuantityTypeIdentifierStepCount", "steps", "count")
resting_hr = extract("HKQuantityTypeIdentifierRestingHeartRate", "resting_hr", "count/min")
vo2        = extract("HKQuantityTypeIdentifierVO2Max", "vo2max", "mL/min·kg")
resp_rate  = extract("HKQuantityTypeIdentifierRespiratoryRate", "respiratory_rate", "count/min")
spo2       = extract("HKQuantityTypeIdentifierOxygenSaturation", "spo2_percent", "%")

# ---------- Energy metrics (now with correct unit 'Cal') ----------
active_energy = extract(
    "HKQuantityTypeIdentifierActiveEnergyBurned",
    "active_energy_cal",
    "Cal"
)

basal_energy = extract(
    "HKQuantityTypeIdentifierBasalEnergyBurned",
    "basal_energy_cal",
    "Cal"
)

# ---------- Distance / movement ----------
distance_walk_run = extract(
    "HKQuantityTypeIdentifierDistanceWalkingRunning",
    "distance_walk_run_m",
    None  # unit often 'm'; leaving None to be less fragile
)

flights = extract(
    "HKQuantityTypeIdentifierFlightsClimbed",
    "flights",
    "count"
)

exercise_time = extract(
    "HKQuantityTypeIdentifierAppleExerciseTime",
    "exercise_minutes",
    "min"
)

stand_time = extract(
    "HKQuantityTypeIdentifierAppleStandTime",
    "stand_minutes",
    "min"
)

# ---------- Sleeping wrist temperature (correct type + unit) ----------
wrist_temp_sleep = extract(
    "HKQuantityTypeIdentifierAppleSleepingWristTemperature",
    "sleeping_wrist_temp_degF",
    "degF"
)

# ---------- Sleep (special handling) ----------
sleep = df[df["type"] == "HKCategoryTypeIdentifierSleepAnalysis"].copy()
if not sleep.empty:
    sleep["sleep_minutes"] = (
        pd.to_datetime(sleep["endDate"]) - pd.to_datetime(sleep["startDate"])
    ).dt.total_seconds() / 60
    sleep = sleep[["startDate", "sleep_minutes"]].rename(columns={"startDate": "timestamp"})
else:
    sleep = pd.DataFrame(columns=["timestamp", "sleep_minutes"])


# ---------- Merge everything into ONE DataFrame ----------
metric_dfs = [
    heart_rate, hrv, steps, resting_hr, vo2,
    resp_rate, spo2,
    active_energy, basal_energy,
    distance_walk_run, flights,
    exercise_time, stand_time,
    wrist_temp_sleep
]

metric_dfs = [m for m in metric_dfs if not m.empty]

if not metric_dfs:
    raise ValueError("No metric dataframes found; check your health_records types.")

merged = metric_dfs[0]
for m in metric_dfs[1:]:
    merged = merged.merge(m, on="startDate", how="outer")

# Sort and rename time column
merged = merged.sort_values("startDate")
merged = merged.rename(columns={"startDate": "timestamp"})

# Merge sleep info
merged = merged.merge(sleep, on="timestamp", how="left")

# Clean
merged = merged.sort_values("timestamp")
merged = merged.ffill().bfill()  # optional

out_path = base_dir / "apple_watch_combined_full.csv"
merged.to_csv(out_path, index=False)

print("\nCombined full CSV saved at:")
print(out_path)
print("\nColumns in final dataset:")
print(merged.columns.tolist())
print("\nPreview:")
print(merged.head())


Loading: /Users/nilvaghela/Desktop/Research/Diabetes/Data/health_records.csv


/var/folders/ph/wydd_qxn59v1_1ynz85860l00000gn/T/ipykernel_26906/3988047346.py:14: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df["endDate"]   = pd.to_datetime(df["endDate"], errors="ignore")



Combined full CSV saved at:
/Users/nilvaghela/Desktop/Research/Diabetes/Data/apple_watch_combined_full.csv

Columns in final dataset:
['timestamp', 'heart_rate', 'hrv', 'steps', 'resting_hr', 'vo2max', 'respiratory_rate', 'spo2_percent', 'active_energy_cal', 'basal_energy_cal', 'distance_walk_run_m', 'flights', 'exercise_minutes', 'stand_minutes', 'sleeping_wrist_temp_degF', 'sleep_minutes']

Preview:
                  timestamp  heart_rate      hrv  steps  resting_hr  vo2max  \
0 2018-02-10 05:37:30-05:00        79.0  132.015   13.0        70.0   34.25   
1 2018-02-10 06:34:33-05:00        79.0  132.015  355.0        70.0   34.25   
2 2018-02-10 06:44:32-05:00        79.0  132.015  157.0        70.0   34.25   
3 2018-02-10 06:54:19-05:00        79.0  132.015   24.0        70.0   34.25   
4 2018-02-10 07:07:11-05:00        79.0  132.015   38.0        70.0   34.25   

   respiratory_rate  spo2_percent  active_energy_cal  basal_energy_cal  \
0              20.5          0.99            

In [25]:
import pandas as pd
from pathlib import Path

base_dir = Path("/Users/nilvaghela/Desktop/Research/Diabetes/Data")
combined_path = base_dir / "apple_watch_combined_full.csv"

df = pd.read_csv(combined_path)

# Parse timestamp and extract date
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["date"] = df["timestamp"].dt.date

# Define aggregations per day
agg_dict = {
    "heart_rate": ["mean", "min", "max", "std"],
    "hrv": ["mean", "min", "max", "std"],
    "steps": ["sum", "max"],
    "resting_hr": ["mean"],
    "vo2max": ["mean"],
    "respiratory_rate": ["mean", "min", "max"],
    "spo2_percent": ["mean", "min"],
    "active_energy_cal": ["sum"],
    "basal_energy_cal": ["sum"],
    "distance_walk_run_m": ["sum"],
    "flights": ["sum"],
    "exercise_minutes": ["sum"],
    "stand_minutes": ["sum"],
    "sleeping_wrist_temp_degF": ["mean"],
    "sleep_minutes": ["sum"]
}

daily = df.groupby("date").agg(agg_dict)

# Flatten MultiIndex columns: ('heart_rate','mean') -> 'heart_rate_mean'
daily.columns = ["{}_{}".format(col[0], col[1]) for col in daily.columns]
daily = daily.reset_index()

out_path = base_dir / "apple_watch_daily_features.csv"
daily.to_csv(out_path, index=False)

print("Saved daily features to:", out_path)
print(daily.head())


Saved daily features to: /Users/nilvaghela/Desktop/Research/Diabetes/Data/apple_watch_daily_features.csv
         date  heart_rate_mean  heart_rate_min  heart_rate_max  \
0  2018-02-10             79.0            79.0            79.0   
1  2018-02-11             79.0            79.0            79.0   
2  2018-02-12             79.0            79.0            79.0   
3  2018-02-13             79.0            79.0            79.0   
4  2018-02-14             79.0            79.0            79.0   

   heart_rate_std  hrv_mean  hrv_min  hrv_max  hrv_std  steps_sum  ...  \
0             0.0   132.015  132.015  132.015      0.0     1980.0  ...   
1             0.0   132.015  132.015  132.015      0.0     4239.0  ...   
2             0.0   132.015  132.015  132.015      0.0     4905.0  ...   
3             0.0   132.015  132.015  132.015      0.0     6242.0  ...   
4             0.0   132.015  132.015  132.015      0.0     5133.0  ...   

   spo2_percent_mean  spo2_percent_min  active_energy

In [26]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta

# Where to save all the CSVs
OUTPUT_DIR = "/Users/nilvaghela/Desktop/Research/Diabetes/Patients_90days"
os.makedirs(OUTPUT_DIR, exist_ok=True)

NUM_DIABETIC = 30
NUM_NORM = 20
N_DAYS = 90  # 90 days of data per patient


def generate_random_day(is_diabetic, day_offset):
    """
    Generate one day's synthetic Apple Watch daily features
    for a single patient.
    """
    day_date = date.today() - timedelta(days=day_offset)

    if is_diabetic:
        hr_mean = np.random.uniform(95, 120)
        hrv_mean = np.random.uniform(20, 60)
        steps = np.random.uniform(1500, 6000)
        resting_hr = np.random.uniform(75, 95)
        spo2 = np.random.uniform(94, 98)
        sleep_temp = np.random.uniform(96.8, 98.4)
        resp_mean = np.random.uniform(19, 26)
        ex_minutes = np.random.uniform(5, 30)
        stand_minutes = np.random.uniform(20, 55)
    else:
        hr_mean = np.random.uniform(65, 85)
        hrv_mean = np.random.uniform(90, 160)
        steps = np.random.uniform(6000, 14000)
        resting_hr = np.random.uniform(58, 72)
        spo2 = np.random.uniform(97, 100)
        sleep_temp = np.random.uniform(95.8, 97.4)
        resp_mean = np.random.uniform(16, 20)
        ex_minutes = np.random.uniform(20, 75)
        stand_minutes = np.random.uniform(40, 80)

    row = {
        "date": day_date,
        "heart_rate_mean": hr_mean,
        "heart_rate_min": hr_mean - np.random.uniform(2, 8),
        "heart_rate_max": hr_mean + np.random.uniform(8, 20),
        "heart_rate_std": np.random.uniform(3, 10),

        "hrv_mean": hrv_mean,
        "hrv_min": max(0, hrv_mean - np.random.uniform(5, 30)),
        "hrv_max": hrv_mean + np.random.uniform(10, 30),
        "hrv_std": np.random.uniform(5, 20),

        "steps_sum": steps,
        "steps_max": np.random.uniform(300, 700),

        "resting_hr_mean": resting_hr,
        "vo2max_mean": np.random.uniform(28, 45),

        "respiratory_rate_mean": resp_mean,
        "respiratory_rate_min": resp_mean - np.random.uniform(1, 3),
        "respiratory_rate_max": resp_mean + np.random.uniform(2, 5),

        "spo2_percent_mean": spo2,
        "spo2_percent_min": spo2 - np.random.uniform(0.1, 0.7),

        "active_energy_cal_sum": np.random.uniform(150, 800),
        "basal_energy_cal_sum": np.random.uniform(1400, 1900),

        "distance_walk_run_m_sum": steps * np.random.uniform(0.6, 1.1) / 1.3,
        "flights_sum": np.random.uniform(5, 40),

        "exercise_minutes_sum": ex_minutes,
        "stand_minutes_sum": stand_minutes,

        "sleeping_wrist_temp_degF_mean": sleep_temp,
        "sleep_minutes_sum": np.random.uniform(0.3, 0.95),  # fraction of 24h like your data

        "diabetes": 1 if is_diabetic else 0,
    }

    return row


# ------------- Generate CSVs ------------- #

# Diabetic patients first: Patient_1_d ... Patient_30_d
for pid in range(1, NUM_DIABETIC + 1):
    rows = []
    for d in range(N_DAYS):
        rows.append(generate_random_day(is_diabetic=True, day_offset=d))
    df = pd.DataFrame(rows)
    df = df.sort_values("date")
    file_path = os.path.join(OUTPUT_DIR, f"Patient_{pid}_d.csv")
    df.to_csv(file_path, index=False)

# Non-diabetic patients: Patient_31 ... Patient_50
for pid in range(NUM_DIABETIC + 1, NUM_DIABETIC + NUM_NORM + 1):
    rows = []
    for d in range(N_DAYS):
        rows.append(generate_random_day(is_diabetic=False, day_offset=d))
    df = pd.DataFrame(rows)
    df = df.sort_values("date")
    file_path = os.path.join(OUTPUT_DIR, f"Patient_{pid}.csv")
    df.to_csv(file_path, index=False)

print("DONE — created 50 patients × 90 days each.")
print("Output folder:", OUTPUT_DIR)


DONE — created 50 patients × 90 days each.
Output folder: /Users/nilvaghela/Desktop/Research/Diabetes/Patients_90days
